In [2]:
pip install scikit-learn

  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   ------- -------------------------------- 1.6/8.9 MB 10.5 MB/s eta 0:00:01
   ----------------- ---------------------- 3.9/8.9 MB 11.2 MB/s eta 0:00:01
   ---------------------------- ----------- 6.3/8.9 MB 11.3 MB/s eta 0:00:01
   -------------------------------------- - 8.7/8.9 MB 11.4 MB/s eta 0:00:01
   ---------------------------------------- 8.9/8.9 MB 11.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/38.6 MB ? eta -:--:--
   -- ------------------------------------- 2.4/38.6 MB 12.2 MB/s eta 0:00:03
   ---- ----------------------------------- 4.7/38.6 MB 11.9 MB/s eta 0:00:03
   ------- -------------------------------- 7.3/38.6 MB 11.6 MB/s eta 0:00:03
   ---------- ----------------------------- 10.0/38.6 MB 11.7 MB/s eta 0:00:03
   ------------ --------------------------- 12.3/38.6 MB 11.7 MB/s eta 0:00:03
   ------------

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
import os

In [2]:
# Make directories for processed data
os.makedirs('data/interim', exist_ok=True)
os.makedirs('data/processed', exist_ok=True)
os.makedirs('models', exist_ok=True)

df = pd.read_csv('data/raw/diabetes.csv')
print(df.shape)
df.head()

(768, 9)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
cols_with_zeros = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
# Replace zero with np.nan
df[cols_with_zeros] = df[cols_with_zeros].replace(0, np.nan)

for col in cols_with_zeros:
    df[col].fillna(df[col].median(), inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_16664\2409198158.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)
C:\Users\USER\AppData\Local\Temp\ipykernel_16664\2409198158.py:6: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example

In [4]:
print(df.shape)
df.head()

(768, 9)


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148.0,72.0,35.0,125.0,33.6,0.627,50,1
1,1,85.0,66.0,29.0,125.0,26.6,0.351,31,0
2,8,183.0,64.0,29.0,125.0,23.3,0.672,32,1
3,1,89.0,66.0,23.0,94.0,28.1,0.167,21,0
4,0,137.0,40.0,35.0,168.0,43.1,2.288,33,1


In [8]:
#BMI category and Age group 

def bmi_category(bmi):
    if bmi < 18.5:
        return 'Underweight'
    elif 18.5 <= bmi < 25:
        return 'Normal'
    elif 25 <= bmi < 30:
        return 'Overweight'
    else:
        return 'Obese'

df['BMI_Category'] = df['BMI'].apply(bmi_category)

def age_group(age):
    if age < 30:
        return 'Young'
    elif age < 50:
        return 'Middle'
    else:
        return 'Senior'

df['Age_Group'] = df['Age'].apply(age_group)

In [9]:
df = pd.get_dummies(df, columns=['BMI_Category', 'Age_Group'], drop_first=True)

In [ ]:
#Test the outcome 

X = df.drop('Outcome', axis=1)
y = df['Outcome']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [11]:
#scaling features for data leakage

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Save the scaler for future inference/deployment
joblib.dump(scaler, 'models/standard_scaler.pkl')

# Convert back to DataFrames to retain column names
X_train_scaled = pd.DataFrame(X_train_scaled, columns=X.columns, index=X_train.index)
X_test_scaled  = pd.DataFrame(X_test_scaled,  columns=X.columns, index=X_test.index)

In [12]:
#cleaned/processed

X_train_scaled['Outcome'] = y_train
X_test_scaled['Outcome'] = y_test

X_train_scaled.to_csv('data/processed/diabetes_train_processed.csv', index=False)
X_test_scaled.to_csv('data/processed/diabetes_test_processed.csv', index=False)